In [1]:
import pandas as pd
import plotly.graph_objects as go
import kagglehub
import os
import glob

# --- 1. DATA ACQUISITION & LOADING ---
# Download the latest US DOT Flight Delays dataset
path = kagglehub.dataset_download("usdot/flight-delays")
flights_file = glob.glob(os.path.join(path, "**/flights.csv"), recursive=True)[0]

# Loading specific columns needed for volume and congestion analysis
# YEAR, MONTH, DAY for date; ORIGIN_AIRPORT for hub analysis
cols = ['YEAR', 'MONTH', 'DAY', 'ORIGIN_AIRPORT']
df = pd.read_csv(flights_file, usecols=cols, low_memory=False)

# --- 2. DATA PRE-PROCESSING ---
# Creating a unified DATE column
df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])

# HUB List and October Numeric DOT ID Mappings
target_hubs = ['ATL', 'ORD', 'DFW', 'DEN', 'LAX', 'SFO', 'PHX', 'IAH', 'LAS', 'MSP']
dot_mapping = {
    '10397': 'ATL', '13930': 'ORD', '11298': 'DFW', '11292': 'DEN',
    '12892': 'LAX', '14771': 'SFO', '14107': 'PHX', '12266': 'IAH',
    '12889': 'LAS', '13487': 'MSP'
}

def generate_congestion_dashboard(df_input):
    # Normalize DOT IDs to IATA codes (Fixing the October Anomaly)
    df_clean = df_input.copy()
    df_clean['ORIGIN_AIRPORT'] = df_clean['ORIGIN_AIRPORT'].astype(str).replace(dot_mapping)

    # Filter only for the selected major hubs
    df_filtered = df_clean[df_clean['ORIGIN_AIRPORT'].isin(target_hubs)]

    # Calculate 50th Percentile (Median) Threshold for each Hub to define "Congestion"
    daily_counts = df_filtered.groupby(['ORIGIN_AIRPORT', 'DATE']).size().reset_index(name='Daily_Count')
    hub_thresholds = daily_counts.groupby('ORIGIN_AIRPORT')['Daily_Count'].quantile(0.50).to_dict()

    fig = go.Figure()

    # Create bar traces for each airport
    for airport in target_hubs:
        airport_data = df_filtered[df_filtered['ORIGIN_AIRPORT'] == airport]

        # Calculate monthly average of daily flight volumes
        monthly_stats = airport_data.groupby(['MONTH', 'DATE']).size().reset_index(name='Count')
        monthly_avg = monthly_stats.groupby('MONTH')['Count'].mean().reset_index()

        # Ensure all 12 months are present in the visualization
        all_months = pd.DataFrame({'MONTH': range(1, 13)})
        monthly_avg = all_months.merge(monthly_avg, on='MONTH', how='left').fillna(0)

        threshold = hub_thresholds[airport]

        # Color Logic: Red (Congested > Median), Green (Normal <= Median)
        colors = ['#EF553B' if val > threshold else '#00CC96' for val in monthly_avg['Count']]

        fig.add_trace(
            go.Bar(
                x=monthly_avg['MONTH'],
                y=monthly_avg['Count'],
                marker_color=colors,
                name=airport,
                visible=(airport == target_hubs[0]), # Only first hub visible by default
                hovertemplate=(
                    "<b>Month:</b> %{x}<br>" +
                    "<b>Avg Daily Volume:</b> %{y:.2f}<br>" +
                    f"<b>Congestion Threshold:</b> {threshold:.2f}<br>" +
                    "<extra></extra>"
                )
            )
        )

    # --- 3. INTERACTIVE DROPDOWN MENU ---
    buttons = []
    for i, airport in enumerate(target_hubs):
        # Create a visibility mask for the selected airport
        visibility = [False] * len(target_hubs)
        visibility[i] = True

        buttons.append(dict(
            label=airport,
            method="update",
            args=[{"visible": visibility},
                  {"title": f"Monthly Operational Congestion: {airport}"}]
        ))

    # --- 4. LAYOUT DESIGN ---
    fig.update_layout(
        updatemenus=[dict(active=0, buttons=buttons, x=1.15, y=1.2)],
        title=f"Monthly Operational Congestion: {target_hubs[0]}",
        xaxis=dict(
            title="Months",
            tickmode='array',
            tickvals=list(range(1, 13)),
            ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        ),
        yaxis=dict(title="Average Daily Flight Volume"),
        template="plotly_white",
        showlegend=False,
        margin=dict(t=100, b=50, l=50, r=100)
    )

    # Save as interactive HTML file
    fig.write_html("airline_congestion_dashboard.html", include_plotlyjs='cdn')

    # Display the final interactive dashboard
    fig.show()

# Execute the dashboard generation
generate_congestion_dashboard(df)

100%|██████████| 191M/191M [00:02<00:00, 67.0MB/s]

Extracting files...
